In [169]:
import numpy as np
import pandas as pd
from scipy import stats

songs=pd.read_csv('data/tracks.csv')
songs=songs.head(100000)

In [170]:
songs_for_linear_regression = songs.copy()
songs_for_linear_regression.set_index('id',inplace=True)

In [171]:
def drop_columns(dataframe,to_be_deleted):
    dataframe.drop(to_be_deleted, axis=1, inplace=True)
to_be_deleted = ['id_artists', 'artists','name','release_date'] 

In [172]:
songs_for_linear_regression['release_date'] = pd.to_datetime(songs_for_linear_regression['release_date'], errors='coerce')

In [173]:
songs_for_linear_regression['year'] = pd.DatetimeIndex(songs_for_linear_regression['release_date']).year
songs_for_linear_regression['month'] = pd.DatetimeIndex(songs_for_linear_regression['release_date']).month


In [174]:
import sklearn
from sklearn import preprocessing

def scaler(dataframe,to_be_scaled):
    scaler = sklearn.preprocessing.StandardScaler(copy = True)
    dataframe[to_be_scaled] = scaler.fit_transform(dataframe[to_be_scaled].to_numpy())
# to_be_scaled = ['popularity','danceability','energy','key','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature']
to_be_scaled = ['loudness','duration_ms','tempo']

In [175]:
drop_columns(songs_for_linear_regression,to_be_deleted)#delete columns
songs_for_linear_regression["key"] = songs_for_linear_regression["key"].astype("category")
songs_for_linear_regression = pd.get_dummies(songs_for_linear_regression, columns=["key"])

In [176]:
# songs_for_linear_regression["year"] = songs_for_linear_regression["year"].astype("category")
# songs_for_linear_regression = pd.get_dummies(songs_for_linear_regression, columns=["year"])

In [177]:
# songs_for_linear_regression["month"] = songs_for_linear_regression["month"].astype("category")
# songs_for_linear_regression = pd.get_dummies(songs_for_linear_regression, columns=["month"])

In [178]:
songs_for_linear_regression[(np.abs(stats.zscore(songs_for_linear_regression)) < 5).all(axis=1)]

In [179]:
scaler(songs_for_linear_regression,to_be_scaled)

In [180]:
songs_for_linear_regression.dropna(inplace=True)

In [181]:
x = songs_for_linear_regression['valence']
del songs_for_linear_regression["valence"]
songs_for_linear_regression['valence'] = x

# AND SO IT BEGINS

In [182]:
import statsmodels.api as sm

In [183]:
# independent_variables = songs_for_linear_regression.columns.drop('valence')

In [184]:
# y = songs_for_linear_regression['valence']
# X = songs_for_linear_regression[independent_variables]

In [185]:
# X = sm.add_constant(X)
# model11 = sm.OLS(y, X).fit()
# model11.summary() 

# CODE FOR FORWARD SELECTION

# METHODS OF LOURIDAS FOR FORWARD SELECTION

In [186]:
def process_subset(y, data, feature_set):
    X = data.loc[:, feature_set].values
    X = sm.add_constant(X)
    names = ['intercept']
    names.extend(feature_set)
    model = sm.OLS(y, X)
    model.data.xnames = names
    regr = model.fit()
    return regr

In [187]:
import itertools

def get_best_of_k(y, data, k):
    
    best_rsquared = 0
    best_model = None
    for comb in itertools.combinations(data.columns, k):
        regr = process_subset(y, data, comb)
        if regr.rsquared > best_rsquared:
            best_rsquared = regr.rsquared
            best_model = regr

    return best_model

In [188]:
def best_subset_selection(data, exog):
    best_model = None
    best_models = []
    y = data.loc[:, exog]
    endog = [ x for x in data.columns if x != exog ]
    X = data.loc[:, endog]

    for i in range(1, len(data.columns)):
        print(f'Finding the best model for {i} variable{"s" if i > 1 else ""}')
        model = get_best_of_k(y, X, i)
        if not best_model or model.rsquared_adj > best_model.rsquared_adj:
            best_model = model
        print(model.model.data.xnames[1:]) # get the variables minums the intercept
        best_models.append(model)

    print(f'Fitted {2**len(data.columns)} models')
    return best_model, best_models

In [189]:
def forward_add_variable(data, exog, selected, to_select):
    best_rsquared = 0
    best_model = None
    best_column = None
    y = data.loc[:, exog]
    
    for column in to_select:
        new_selected = selected + [column]
        regr = process_subset(y, data, new_selected)
        if regr.rsquared > best_rsquared:
            best_rsquared = regr.rsquared
            best_model = regr
            best_column = column
    
    return best_model, best_column

In [190]:
def forward_stepwise_selection(data, exog):

    best_models = []
    best_model = None
    selected = []
    to_select = [ x for x in data.columns if x != exog ]

    p = len(to_select) + 1

    for i in range(1, p):
        print(f'Finding the best model for {i} variable{"s" if i > 1 else ""}')
        model, best_column = forward_add_variable(data, exog, selected, to_select)
        selected.append(best_column)
        to_select.remove(best_column)
        if not best_model or model.rsquared_adj > best_model.rsquared_adj:
            best_model = model
        print(selected)
        best_models.append(model)
        print(model.rsquared_adj)
        
    print(f'Fitted {1 + p*(p+1)//2} models')
    return best_model, best_models

In [191]:
best_model, _ = forward_stepwise_selection(songs_for_linear_regression, 'valence')
print('Best overall model:', len(best_model.model.exog_names), best_model.model.exog_names)

Finding the best model for 1 variable
['danceability']
0.3124838182838984
Finding the best model for 2 variables
['danceability', 'energy']
0.365970471384645
Finding the best model for 3 variables
['danceability', 'energy', 'year']
0.4509446454436268
Finding the best model for 4 variables
['danceability', 'energy', 'year', 'tempo']
0.4648999590185098
Finding the best model for 5 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms']
0.47691383577569546
Finding the best model for 6 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms', 'speechiness']
0.48856290574579175
Finding the best model for 7 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms', 'speechiness', 'acousticness']
0.4951361512433875
Finding the best model for 8 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms', 'speechiness', 'acousticness', 'explicit']
0.5004235989657495
Finding the best model for 9 variables
['danceability', 'energy', 'year', 'tempo', 'dura

In [192]:
best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                valence   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     4470.
Date:                Fri, 11 Feb 2022   Prob (F-statistic):               0.00
Time:                        00:43:42   Log-Likelihood:                 27711.
No. Observations:              100000   AIC:                        -5.537e+04
Df Residuals:                   99976   BIC:                        -5.514e+04
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
intercept            5.6391      0.063     88.891      0.000       5.515       5.763
danceability         0.8397      0.004    217.245      0.000       0.832       0.847
energy               0.5404      0.005    114.470      0.000       0.531       0.550
year                -0.0030   3.19e-05    -93.156      0.000      -0.003      -0.003
tempo                0.0311      0.001     51.406      0.000       0.030       0.032
duration_ms         -0.0271      0.001    -45.653      0.000      -0.028      -0.026
speechiness         -0.1515      0.004    -39.777      0.000      -0.159      -0.144
acousticness         0.1163      0.003     41.203      0.000       0.111       0.122
explicit            -0.0965      0.003    -30.413      0.000      -0.103      -0.090
instrumentalness    -0.0495      0.002    -24.123      0.000      -0.054      -0.045
loudness            -0.0216      0.001    -20.972      0.000      -0.024      -0.020
mode                 0.0181      0.001     14.026      0.000       0.016       0.021
popularity           0.0002   3.24e-05      6.743      0.000       0.000       0.000
key_9                0.0106      0.002      4.970      0.000       0.006       0.015
key_5                0.0093      0.002      4.318      0.000       0.005       0.014
key_7                0.0053      0.002      2.631      0.009       0.001       0.009
key_2                0.0043      0.002      2.065      0.039       0.000       0.008
liveness             0.0104      0.003      3.016      0.003       0.004       0.017
key_1               -0.0101      0.002     -4.219      0.000      -0.015      -0.005
key_6               -0.0105      0.003     -3.737      0.000      -0.016      -0.005
key_8               -0.0086      0.003     -3.341      0.001      -0.014      -0.004
time_signature       0.0018      0.001      1.572      0.116      -0.000       0.004
key_3               -0.0046      0.003     -1.574      0.115      -0.010       0.001
key_11              -0.0031      0.003     -1.172      0.241      -0.008       0.002
==============================================================================
Omnibus:                      288.260   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              290.941
Skew:                          -0.130   Prob(JB):                     6.65e-64
Kurtosis:                       3.048   Cond. No.                     2.16e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""